## Preprocessing of raster images using Python (version 3.5): Example using flight of _Acacia dealbata_ 

## Processing steps include:
### - 1: mask images using GDAL utils
### - 2: resample a 'slave' raster using a 'master' raster, using GDAL python library

In [12]:
# import libraries
import os, glob

In [13]:
# prepare directories
inputRasterDir = "H:/Teja/UAV_mapping_invasive"
outRawDir = "F:/invasive_spp/raw/accacia_f1/"
os.chdir(inputRasterDir)

In [14]:
# list files
hyper = inputRasterDir+"/hyper/acacia_f1/Hyper_A1_biobio1.tif"
rgb = inputRasterDir+"/rgb/acacia_f1_rgb/flight_01_biobio_ortho003_19s.tif"
structure = "/structure/biobio_f1_xyz.tif"

In [15]:
# textures
os.chdir(inputRasterDir+"/texture")
texture = glob.glob("flight_01_biobio*.tif")
texture = ["H:/Teja/UAV_mapping_invasive/texture/" + s for s in texture]
print(texture)
print("")
print("list length :", len(texture))

['H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_0.25.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_0.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_0.75.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_1.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_1.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_2.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_2.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_3.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_3.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_4.tif']

list length : 10


In [16]:
# append all data
list_all = [hyper, rgb, structure] + texture
print(list_all)
print("")
print("list length :", len(list_all))

['H:/Teja/UAV_mapping_invasive/hyper/acacia_f1/Hyper_A1_biobio1.tif', 'H:/Teja/UAV_mapping_invasive/rgb/acacia_f1_rgb/flight_01_biobio_ortho003.tif', '/structure/biobio_f1_xyz.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_0.25.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_0.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_0.75.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_1.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_1.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_2.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_2.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_3.5.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_3.tif', 'H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_4.tif']

list length : 13


## Mask rasters using GDAL

In [17]:
# import subprocess to run terminal
from subprocess import call

In [18]:
# shapefile used to crop the rasters
shp = inputRasterDir+"/1_reference/acacia_f1_AOI.shp"

In [19]:
# run gdalwarp for all rasters
for raster in list_all:
    baseName = os.path.basename(raster)
    process = "gdalwarp -dstnodata -9999 -cutline "+shp+" "+raster+" "+outRawDir+baseName[:-4]+ "_clip.tif"
    call(process)
    print("mask for :", baseName)

mask for : Hyper_A1_biobio1.tif
mask for : flight_01_biobio_ortho003.tif
mask for : biobio_f1_xyz.tif
mask for : flight_01_biobio_ortho003_0.25.tif
mask for : flight_01_biobio_ortho003_0.5.tif
mask for : flight_01_biobio_ortho003_0.75.tif
mask for : flight_01_biobio_ortho003_1.5.tif
mask for : flight_01_biobio_ortho003_1.tif
mask for : flight_01_biobio_ortho003_2.5.tif
mask for : flight_01_biobio_ortho003_2.tif
mask for : flight_01_biobio_ortho003_3.5.tif
mask for : flight_01_biobio_ortho003_3.tif
mask for : flight_01_biobio_ortho003_4.tif


In [23]:
print(process)

gdalwarp -dstnodata -9999 -cutline H:/Teja/UAV_mapping_invasive/1_reference/acacia_f1_AOI.shp H:/Teja/UAV_mapping_invasive/texture/flight_01_biobio_ortho003_4.tif F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_4_clip.tif


## Resample rasters to match the hyperspectral resolution and extention

In [20]:
# function to resample the "slave" raster using a "master" raster
def reproject_image_to_master( master, slave, res=None ):
    import gdal
    
    slave_ds = gdal.Open( slave )
    if slave_ds is None:
        raise IOError
    slave_proj = slave_ds.GetProjection()
    slave_geotrans = slave_ds.GetGeoTransform()
    data_type = slave_ds.GetRasterBand(1).DataType
    n_bands = slave_ds.RasterCount

    master_ds = gdal.Open( master )
    if master_ds is None:
        raise IOError
    master_proj = master_ds.GetProjection()
    master_geotrans = master_ds.GetGeoTransform()
    w = master_ds.RasterXSize
    h = master_ds.RasterYSize
    if res is not None:
        master_geotrans[1] = float( res )
        master_geotrans[-1] = - float ( res )

    dst_filename = slave.replace( ".tif", "_crop.tif" )
    dst_ds = gdal.GetDriverByName('GTiff').Create(dst_filename, w, h, n_bands, data_type)
    dst_ds.SetGeoTransform( master_geotrans )
    dst_ds.SetProjection( master_proj)

    gdal.ReprojectImage( slave_ds, dst_ds, slave_proj,
                         master_proj, gdal.GRA_NearestNeighbour)
    dst_ds = None  # Flush to disk
    return dst_filename

In [21]:
# output direction
os.chdir(outRawDir)

# list of all tiffs
list_all = glob.glob("*.tif")
list_all = [outRawDir + s for s in list_all]
# add folder dir
print(list_all)
print("")
print("length of the list:", len(list_all))

['F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_0.25_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_0.5_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_0.75_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_1.5_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_1_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_2.5_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_2_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_3.5_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_3_clip.tif', 'F:/invasive_spp/raw/accacia_f1/flight_01_biobio_ortho003_4_clip.tif', 'F:/invasive_spp/raw/accacia_f1/Hyper_A1_biobio1_clip.tif']

length of the list: 11


In [22]:
# resample
for raster in list_all[1:]:
    baseName = os.path.basename(raster)
    reproject_image_to_master(hyper, raster) 
    # delete the file in the raw folder
    os.remove(raster)
    print("mask for :", baseName)

mask for : flight_01_biobio_ortho003_0.5_clip.tif
mask for : flight_01_biobio_ortho003_0.75_clip.tif
mask for : flight_01_biobio_ortho003_1.5_clip.tif
mask for : flight_01_biobio_ortho003_1_clip.tif
mask for : flight_01_biobio_ortho003_2.5_clip.tif
mask for : flight_01_biobio_ortho003_2_clip.tif
mask for : flight_01_biobio_ortho003_3.5_clip.tif
mask for : flight_01_biobio_ortho003_3_clip.tif
mask for : flight_01_biobio_ortho003_4_clip.tif
mask for : Hyper_A1_biobio1_clip.tif
